# This Pyhton scripts reads the data.h5 saved in Kapsel and generates the .gsd data of the squirmers to mount movies in Ovito

# Author: Federico Fadda
# Date: 5/12/2018

In [7]:
import numpy as np
import gsd.hoomd
import h5py

In [16]:
#Reading the data in the data.h5 file after the simulation:
#Hoomd schema is used:

rf = "/Users/federico/mnt/ebi/Shear_azimuthal/0/Shear_1.1_x/output/data.h5"
f = h5py.File(rf, 'r')

Lx=64
Ly=64
Lz=64
Np=1 #number of particles in the system
N1=1
N2=0
tmax=200 #number of snapshots chosen in the input file of Kapsel

VCM=np.zeros(((tmax,Np,3)))
CM=np.zeros(((tmax,Np,3)))
CMX=np.zeros(Np)
CMY=np.zeros(Np)
CMZ=np.zeros(Np)
VCMX=np.zeros(Np)
VCMY=np.zeros(Np)
VCMZ=np.zeros(Np)

#To mount a movie of the squirmers in Ovito the position of each particle in the box is requested!
# TO DO: reading the orientations of the particles!!!

def create_frame(i):
        s = gsd.hoomd.Snapshot()
        s.particles.N = Np
        s.configuration.step = i
        s.configuration.box = [Lx-1, Ly-1, Lz-1, 0, 0, 0]
        s.particles.types = ['0']
        #s.particles.types = ['0', '1']
        s.particles.typeid = np.concatenate(([0 for n in range(0,N1)], [1 for n in range(N1,Np)]))
        CM[i,:,:]=f['trajectory_data/frame_'+str(i)+'/particle/pos'].value 
        VCM[i,:,:]=f['trajectory_data/frame_'+str(i)+'/particle/vel'].value 
        for n in range (0,Np):
            CMX[n]=CM[i,n,0]-(Lx-1)*0.5 #If I put Lx it doesn't work for one particle!
            CMY[n]=CM[i,n,1]-(Ly-1)*0.5
            CMZ[n]=CM[i,n,2]-(Lz-1)*0.5
            VCMX[n]=VCM[i,n,0]
            VCMY[n]=VCM[i,n,1]
            VCMZ[n]=VCM[i,n,2]
        s.orientation = [[1,0,0,0]]
        s.particles.position = [[CMX[n],CMY[n],CMZ[n]] for n in range(Np)]
        s.particles.velocity = [[VCMX[n],VCMY[n],VCMZ[n]] for n in range(Np)]
        return s
    
t = gsd.hoomd.open(name='squirmer_movie.gsd', mode='wb')
t.extend( (create_frame(i) for i in range(tmax)) )

